In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Use Gemini to develop Model Context Protocol (MCP) Server



<table align="left">
<td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/mcp/build_mcp_server_by_gemini.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fmcp%2Fbuild_mcp_server_by_gemini.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/mcp/build_mcp_server_by_gemini.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  
  
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/mcp/build_mcp_server_by_gemini.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/mcp/build_mcp_server_by_gemini.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/mcp/build_mcp_server_by_gemini.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/mcp/build_mcp_server_by_gemini.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/mcp/build_mcp_server_by_gemini.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/mcp/build_mcp_server_by_gemini.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Author |
| --- |
| [Dave Wang](https://github.com/wadave) |

## Overview
The Model Context Protocol (MCP) is an open standard that simplifies how AI assistants connect with external data, tools, and systems. It achieves this by standardizing the way applications provide contextual information to Large Language Models (LLMs), creating a vital interface for models to interact directly with various external services.

Developers building MCP-enabled applications have the flexibility to utilize existing third-party MCP servers or implement their own custom server solutions.

This notebook focuses on the latter, demonstrating how to build custom MCP servers using Gemini. We will walk through code generation and testing for three specific examples:

#### MCP server code generation:
- Example 1: Creating a BigQuery MCP Server
- Example 2: Creating a MedlinePlus MCP Server
- Example 3: Creating an NIH MCP Server

#### MCP server code testing:
- Option 1: Use LangChain MCP Adaptor
- Option 2: Build your own agent

## Get started

### Install Google Gen AI SDK and other required packages


In [ ]:
%pip install --upgrade --quiet google-genai google-cloud-secret-manager mcp black google-cloud-bigquery langchain-mcp-adapters langchain langchain-google-vertexai langgraph

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Import Libraries

In [ ]:
import json
import os
import re
import sys
from typing import Any

from IPython.display import Markdown, display
import black
from google import genai
from google.cloud import aiplatform
from google.genai import types
from google.genai.types import (
    GenerateContentConfig,
)
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_google_vertexai import ChatVertexAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import requests

### Helper functions

In [ ]:
def get_url_content(url: str) -> str:
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)

        # Raise an exception if the request returned an error status code (like 404 or 500)
        response.raise_for_status()

        # Get the content of the response as text (HTML, in this case)
        # 'requests' automatically decodes the content based on HTTP headers
        file_content = response.text

        # Now you can work with the content
        print("Successfully fetched content")
        return file_content
        # Or, save it to a file:
        # with open("server_page.html", "w", encoding="utf-8") as f:
        #     f.write(file_content)
        # print("Content saved to server_page.html")

    except requests.exceptions.RequestException as e:
        # Handle potential errors during the request (e.g., network issues, DNS errors)
        print(f"Error fetching URL {url}: {e}")
    except requests.exceptions.HTTPError as e:
        # Handle HTTP error responses (e.g., 404 Not Found, 503 Service Unavailable)
        print(f"HTTP Error for {url}: {e}")


def format_python(raw_code: str, output_filename: str) -> str:

    try:
        # Format the code string using black
        # Use default FileMode which is generally recommended
        formatted_code = black.format_str(raw_code, mode=black.FileMode())

        # Save the formatted code to the specified file
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(formatted_code)

        print(f"Successfully formatted the code and saved it to '{output_filename}'")

    except black.InvalidInput as e:
        print(
            f"Error formatting code: The input string does not seem to be valid Python syntax."
        )
        print(f"Details: {e}")
    except Exception as e:
        print(f"An error occurred while writing the file: {e}")


def extract_json_from_string(input_str: str) -> dict | list | None:
    """
    Extracts JSON data from a string, handling potential variations.

    This function attempts to find JSON data within a string. It specifically
    looks for JSON enclosed in Markdown-like code fences (```json ... ```).
    If such a block is found, it extracts and parses the content.
    If no code block is found, it attempts to parse the entire input string
    as JSON.

    Args:
        input_str: The string potentially containing JSON data. It might be
                   a plain JSON string or contain a Markdown code block
                   with JSON, possibly preceded by other text (like 'shame').

    Returns:
        The parsed JSON object (typically a dictionary or list) if valid
        JSON is found and successfully parsed.
        Returns None if no valid JSON is found, if parsing fails, or if the
        input is not a string.
    """
    if not isinstance(input_str, str):
        # Handle cases where input is not a string
        return None

    # Pattern to find JSON within ```json ... ``` blocks
    # - ````json`: Matches the start fence.
    # - `\s*`: Matches any leading whitespace after the fence marker.
    # - `(.*?)`: Captures the content (non-greedily) between the fences. This is group 1.
    # - `\s*`: Matches any trailing whitespace before the end fence.
    # - ` ``` `: Matches the end fence.
    # - `re.DOTALL`: Allows '.' to match newline characters.
    pattern = r"```json\s*(.*?)\s*```"
    match = re.search(pattern, input_str, re.DOTALL)

    json_string_to_parse = None

    if match:
        # If a markdown block is found, extract its content
        json_string_to_parse = match.group(
            1
        ).strip()  # Get captured group and remove surrounding whitespace
    else:
        # If no markdown block, assume the *entire* input might be JSON
        # We strip whitespace in case the string is just JSON with padding
        json_string_to_parse = input_str.strip()

    if not json_string_to_parse:
        # If after stripping, the potential JSON string is empty, return None
        return None

    try:
        # Attempt to parse the determined string (either from block or whole input)
        parsed_json = json.loads(json_string_to_parse)
        return parsed_json
    except json.JSONDecodeError:
        # Parsing failed, indicating the string wasn't valid JSON
        return None
    except Exception as e:
        # Catch other potential unexpected errors during parsing
        print(f"An unexpected error occurred during JSON parsing: {e}")
        return None

### Set up Vertex AI 

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

## Set up Gemini client

In [ ]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

MODEL_ID = "gemini-2.5-pro"  # This model is only for MCP server code generation.

### Get system instruction context info

In [ ]:
# The URL you want to fetch
url = "https://modelcontextprotocol.io/quickstart/server"
reference_content = get_url_content(url)

### Set up system instruction

In [ ]:
from pydantic import BaseModel


class ResponseSchema(BaseModel):
    python_code: str
    description: str


system_instruction = f"""
  You are an MCP server expert.
  Your mission is to write python code for MCP server.
  Here's the MCP server development guide and example
  {reference_content}
  
"""

#### Define a function to generate MCP server code

In [ ]:
def generate_mcp_server(prompt):
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config=GenerateContentConfig(
            system_instruction=system_instruction,
            response_mime_type="application/json",
            response_schema=ResponseSchema,
        ),
    )

    return response.text

## Generate MCP Server Code

### Example 1:  Create MCP Server for Google Cloud BigQuery

In [ ]:
prompt = """
  Please create an MCP server code for Google Cloud BigQuery. It has two tools. One is to list tables for all datasets, the other is to describe a table. Google Cloud project id and location will be provided for use. please use project id to access BigQuery client.
  Please output JSON output only.
  
"""

In [ ]:
response_text = generate_mcp_server(prompt)

In [ ]:
python_code = extract_json_from_string(response_text)["python_code"]

In [ ]:
format_python(python_code, "server/bq.py")

### Example 2:  Create MCP server for Medlineplus website
Create an MCP server for 
https://medlineplus.gov/about/developers/webservices/ API service

In [ ]:
med_url = "https://medlineplus.gov/about/developers/webservices/"

In [ ]:
prompt_base = """
  Please create an MCP server code for https://medlineplus.gov/about/developers/webservices/. It has one tool, get_medical_term. You provide a medical term, this tool will return explanation of the medial term
  
  Here's the API details:
  
"""

prompt = [prompt_base, types.Part.from_uri(file_uri=med_url, mime_type="text/html")]
response_text = generate_mcp_server(prompt)
python_code = extract_json_from_string(response_text)["python_code"]

format_python(python_code, "server/med.py")

### Example 3: Create MCP Server for NIH

In [ ]:
nih_url = "https://clinicaltables.nlm.nih.gov/apidoc/icd10cm/v3/doc.html"

In [ ]:
prompt_base = """
  Please create an MCP server code for NIH. It has one tool, get_icd_10_code. You provide a name or code, it will return top 5 results. 
  
  Here's the API details:

"""
prompt = [prompt_base, types.Part.from_uri(file_uri=nih_url, mime_type="text/html")]
response_text = generate_mcp_server(prompt)
python_code = extract_json_from_string(response_text)["python_code"]

format_python(python_code, "server/nih.py")

## Testing MCP Servers

### Option 1: Use LangChain MCP adaptor to test MCP servers
It works in Jupyter Notebooks. It may give errors in Colab. Please use option 2 if using Colab. 

In [ ]:
aiplatform.init(
    project=PROJECT_ID,
    location=LOCATION,
)

In [ ]:
llm = ChatVertexAI(
    model="gemini-2.5-pro",
    temperature=0,
    max_tokens=None,
    max_retries=6,
    stop=None,
)

In [ ]:
server_configs = {
    "nih": {
        "command": "python",
        "args": ["./server/nih.py"],
        "transport": "stdio",
    },
    "med": {
        "command": "python",
        "args": ["./server/med.py"],
        "transport": "stdio",
    },
    "bq": {
        "command": "python",
        "args": ["./server/bq.py"],
        "transport": "stdio",
    },
}

Set up MCP Client using LangChain MCP Adaptor 

In [ ]:
async def run_lc_react_agent(server_configs, message):
    async with MultiServerMCPClient(server_configs) as client:
        agent = create_react_agent(llm, client.get_tools())

        agent_response = await agent.ainvoke({"messages": message})
        for response in agent_response["messages"]:
            user = ""

            if isinstance(response, HumanMessage):
                user = "[User]"
            elif isinstance(response, ToolMessage):
                user = "-Tool-"
            elif isinstance(response, AIMessage):
                user = "[Agent]"

            if isinstance(response.content, list):
                display(Markdown(f'{user}: {response.content[0].get("text", "")}'))
                continue
            display(Markdown(f"{user}: {response.content}"))

#### Example 1: BigQuery MCP server testing

In [ ]:
await run_lc_react_agent(
    server_configs, "Please list my BigQuery tables for project 'dw-genai-dev'"
)

#### Example 2: MedlinePlus MCP server testing

In [ ]:
await run_lc_react_agent(server_configs, "Please explain flu in details")

#### Example 3: NIH MCP Server testing

In [ ]:
await run_lc_react_agent(server_configs, "can you tell me icd-10 code for influenza A?")

### Option 2: Build your own agent to test MCP servers

##### Gemini Agent

Within an MCP client session, this agent loop runs a multi-turn conversation loop with a Gemini model, handling tool calls via MCP server.

This function orchestrates the interaction between a user prompt, a Gemini model capable of function calling, and a session object that provides and executes tools. It handles the cycle of:
-  Gemini gets tool information from MCP client session
-  Sending the user prompt (and conversation history) to the model.
-  If the model requests tool calls, Gemini makes initial function calls to get structured data as per schema, and 
-  Sending the tool execution results back to the model.
-  Repeating until the model provides a text response or the maximum number of tool execution turns is reached.
-  Gemini generates final response based on tool responses and original query.
  
MCP integration with Gemini

<img src="https://storage.googleapis.com/github-repo/generative-ai/gemini/mcp/mcp_tool_call.png" alt="MCP with Gemini" height="700">

In [ ]:
# --- Configuration ---
# Consider using a more recent/recommended model if available and suitable

DEFAULT_MAX_TOOL_TURNS = 5  # Maximum consecutive turns for tool execution
DEFAULT_INITIAL_TEMPERATURE = (
    0.0  # Temperature for the first LLM call (more deterministic)
)
DEFAULT_TOOL_CALL_TEMPERATURE = (
    1.0  # Temperature for LLM calls after tool use (potentially more creative)
)

# Make tool calls via MCP Server


async def _execute_tool_calls(
    function_calls: list[types.FunctionCall], session: ClientSession
) -> list[types.Part]:
    """
    Executes a list of function calls requested by the Gemini model via the session.

    Args:
        function_calls: A list of FunctionCall objects from the model's response.
        session: The session object capable of executing tools via `call_tool`.

    Returns:
        A list of Part objects, each containing a FunctionResponse corresponding
        to the execution result of a requested tool call.
    """
    tool_response_parts: list[types.Part] = []
    print(f"--- Executing {len(function_calls)} tool call(s) ---")

    for func_call in function_calls:
        tool_name = func_call.name
        # Ensure args is a dictionary, even if missing or not a dict type
        args = func_call.args if isinstance(func_call.args, dict) else {}
        print(f"  Attempting to call session tool: '{tool_name}' with args: {args}")

        tool_result_payload: dict[str, Any]
        try:
            # Execute the tool using the provided session object
            # Assumes session.call_tool returns an object with attributes
            # like `isError` (bool) and `content` (list of Part-like objects).
            tool_result = await session.call_tool(tool_name, args)
            print(f"  Session tool '{tool_name}' execution finished.")

            # Extract result or error message from the tool result object
            result_text = ""
            # Check structure carefully based on actual `session.call_tool` return type
            if (
                hasattr(tool_result, "content")
                and tool_result.content
                and hasattr(tool_result.content[0], "text")
            ):
                result_text = tool_result.content[0].text or ""

            if hasattr(tool_result, "isError") and tool_result.isError:
                error_message = (
                    result_text
                    or f"Tool '{tool_name}' failed without specific error message."
                )
                print(f"  Tool '{tool_name}' reported an error: {error_message}")
                tool_result_payload = {"error": error_message}
            else:
                print(
                    f"  Tool '{tool_name}' succeeded. Result snippet: {result_text[:150]}..."
                )  # Log snippet
                tool_result_payload = {"result": result_text}

        except Exception as e:
            # Catch exceptions during the tool call itself
            error_message = f"Tool execution framework failed: {type(e).__name__}: {e}"
            print(f"  Error executing tool '{tool_name}': {error_message}")
            tool_result_payload = {"error": error_message}

        # Create a FunctionResponse Part to send back to the model
        tool_response_parts.append(
            types.Part.from_function_response(
                name=tool_name, response=tool_result_payload
            )
        )
    print(f"--- Finished executing tool call(s) ---")
    return tool_response_parts


async def run_agent_loop(
    prompt: str,
    client: genai.Client,
    session: ClientSession,
    model_id: str = MODEL_ID,
    max_tool_turns: int = DEFAULT_MAX_TOOL_TURNS,
    initial_temperature: float = DEFAULT_INITIAL_TEMPERATURE,
    tool_call_temperature: float = DEFAULT_TOOL_CALL_TEMPERATURE,
) -> types.GenerateContentResponse:
    """
    Runs a multi-turn conversation loop with a Gemini model, handling tool calls.

    This function orchestrates the interaction between a user prompt, a Gemini
    model capable of function calling, and a session object that provides
    and executes tools. It handles the cycle of:
    1. Sending the user prompt (and conversation history) to the model.
    2. If the model requests tool calls, executing them via the `session`.
    3. Sending the tool execution results back to the model.
    4. Repeating until the model provides a text response or the maximum
       number of tool execution turns is reached.

    Args:
        prompt: The initial user prompt to start the conversation.
        client: An initialized Gemini GenerativeModel client object

        session: An active session object responsible for listing available tools
                 via `list_tools()` and executing them via `call_tool(tool_name, args)`.
                 It's also expected to have an `initialize()` method.
        model_id: The identifier of the Gemini model to use (e.g., "gemini-2.0-flash").
        max_tool_turns: The maximum number of consecutive turns dedicated to tool calls
                        before forcing a final response or exiting.
        initial_temperature: The temperature setting for the first model call.
        tool_call_temperature: The temperature setting for subsequent model calls
                               that occur after tool execution.

    Returns:
        The final Response from the Gemini model after the
        conversation loop concludes (either with a text response or after
        reaching the max tool turns).

    Raises:
        ValueError: If the session object does not provide any tools.
        Exception: Can potentially raise exceptions from the underlying API calls
                   or session tool execution if not caught internally by `_execute_tool_calls`.
    """
    print(
        f"Starting agent loop with model '{model_id}' and prompt: '{prompt[:100]}...'"
    )

    # Initialize conversation history with the user's prompt
    contents: list[types.Content] = [
        types.Content(role="user", parts=[types.Part(text=prompt)])
    ]

    # Ensure the session is ready (if needed)
    if hasattr(session, "initialize") and callable(session.initialize):
        print("Initializing session...")
        await session.initialize()
    else:
        print("Session object does not have an initialize() method, proceeding anyway.")

    # --- 1. Discover Tools from Session ---
    print("Listing tools from session...")
    # Assumes session.list_tools() returns an object with a 'tools' attribute (list)
    # Each item in the list should have 'name', 'description', and 'inputSchema' attributes.
    session_tool_list = await session.list_tools()

    if not session_tool_list or not session_tool_list.tools:
        raise ValueError("No tools provided by the session. Agent loop cannot proceed.")

    # Convert session tools to the format required by the Gemini API
    gemini_tool_config = types.Tool(
        function_declarations=[
            types.FunctionDeclaration(
                name=tool.name,
                description=tool.description,
                parameters=tool.inputSchema,  # Assumes inputSchema is compatible
            )
            for tool in session_tool_list.tools
        ]
    )
    print(
        f"Configured Gemini with {len(gemini_tool_config.function_declarations)} tool(s)."
    )

    # --- 2. Initial Model Call ---
    print("Making initial call to Gemini model...")
    current_temperature = initial_temperature
    response = await client.aio.models.generate_content(
        model=MODEL_ID,
        contents=contents,  # Send updated history
        config=types.GenerateContentConfig(
            temperature=1.0,
            tools=[gemini_tool_config],
        ),  # Keep sending same config
    )
    print("Initial response received.")

    # Append the model's first response (potentially including function calls) to history
    # Need to handle potential lack of candidates or content
    if not response.candidates:
        print("Warning: Initial model response has no candidates.")
        # Decide how to handle this - raise error or return the empty response?
        return response
    contents.append(response.candidates[0].content)

    # --- 3. Tool Calling Loop ---
    turn_count = 0
    # Check specifically for FunctionCall objects in the latest response part
    latest_content = response.candidates[0].content
    has_function_calls = any(part.function_call for part in latest_content.parts)

    while has_function_calls and turn_count < max_tool_turns:
        turn_count += 1
        print(f"\n--- Tool Turn {turn_count}/{max_tool_turns} ---")

        # --- 3.1 Execute Pending Function Calls ---
        function_calls_to_execute = [
            part.function_call for part in latest_content.parts if part.function_call
        ]
        tool_response_parts = await _execute_tool_calls(
            function_calls_to_execute, session
        )

        # --- 3.2 Add Tool Responses to History ---
        # Send back the results for *all* function calls from the previous turn
        contents.append(
            types.Content(role="function", parts=tool_response_parts)
        )  # Use "function" role
        print(f"Added {len(tool_response_parts)} tool response part(s) to history.")

        # --- 3.3 Make Subsequent Model Call with Tool Responses ---
        print("Making subsequent API call to Gemini with tool responses...")
        current_temperature = tool_call_temperature  # Use different temp for follow-up
        response = await client.aio.models.generate_content(
            model=MODEL_ID,
            contents=contents,  # Send updated history
            config=types.GenerateContentConfig(
                temperature=1.0,
                tools=[gemini_tool_config],
            ),
        )
        print("Subsequent response received.")

        # --- 3.4 Append latest model response and check for more calls ---
        if not response.candidates:
            print("Warning: Subsequent model response has no candidates.")
            break  # Exit loop if no candidates are returned
        latest_content = response.candidates[0].content
        contents.append(latest_content)
        has_function_calls = any(part.function_call for part in latest_content.parts)
        if not has_function_calls:
            print(
                "Model response contains text, no further tool calls requested this turn."
            )

    # --- 4. Loop Termination Check ---
    if turn_count >= max_tool_turns and has_function_calls:
        print(
            f"Maximum tool turns ({max_tool_turns}) reached. Exiting loop even though function calls might be pending."
        )
    elif not has_function_calls:
        print("Tool calling loop finished naturally (model provided text response).")

    # --- 5. Return Final Response ---
    print("Agent loop finished. Returning final response.")
    return response

#### Set up MCP client

In [ ]:
async def run_simple_agent(server_params, query):
    async with stdio_client(server_params) as (
        read,
        write,
    ):
        async with ClientSession(
            read,
            write,
        ) as session:
            # Test prompt
            prompt = query
            print(f"Running agent loop with prompt: {prompt}")
            # Run agent loop
            res = await run_agent_loop(prompt, client, session)
            return res

In [ ]:
bq_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/bq.py"],
)

In [ ]:
med_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/med.py"],
)

In [ ]:
nih_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/nih.py"],
)

In [ ]:
bq_query = (
    "Please list my BigQuery tables, project id is 'dw-genai-dev', location is 'us'"
)
bq_res = await run_simple_agent(bq_server_params, bq_query)
print(bq_res.text)

In [ ]:
med_query = "Please explain flu in detail."
med_res = await run_simple_agent(med_server_params, med_query)
print(med_res.text)

In [ ]:
nih_query = "Please tell me icd-10 code for pneumonia"
nih_res = await run_simple_agent(nih_server_params, nih_query)
print(nih_res.text)